In [1]:
# Instalar biblioteca watermark para visualização de ambiente e versões de pacotes foram usados no projeto
# %pip install -q -U watermark

In [2]:
# Importar bibliotecas
from datetime import datetime

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import joblib  # para salvar o pré-processador se quiser reutilizar

pd.set_option('future.no_silent_downcasting', True)

# Divisão dos Dados 
* Organizar o experimento 

In [3]:
# Carregar base de dados
df=pd.read_csv("heart_disease_uci_tratado_3.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  920 non-null    float64
 6   chol      920 non-null    float64
 7   fbs       920 non-null    bool   
 8   restecg   920 non-null    object 
 9   thalch    920 non-null    float64
 10  exang     920 non-null    bool   
 11  oldpeak   920 non-null    float64
 12  slope     920 non-null    object 
 13  num       920 non-null    int64  
 14  num_reag  920 non-null    int64  
dtypes: bool(2), float64(4), int64(4), object(5)
memory usage: 95.4+ KB


## Separação Treino / Teste

O procedimento mais comum é dividir o dataset em:

* **Treino (Train): 70–80% dos dados**. Usado para ajustar os modelos.
* **Teste (Test): 20–30% dos dados**. Usado para avaliar a performance final do modelo em dados não vistos.

In [5]:
# Variáveis independentes
X = df.drop(columns=["num","num_reag", "id"]) 

In [6]:
# Variáveis alvo
y = df["num_reag"]

In [7]:
# Divisão treino/teste (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

**Observações:**

* **stratify=y** garante que a proporção de classes (presença/ausência de doença) seja mantida em treino e teste.Divide os dados de forma estratificada.
* **random_state** fixa a aleatoriedade para resultados reprodutíveis.
* **test_size=0.2** define que o tamanho de 20% dos dados serão separados para o teste, enquanto os outros 80% serão usados para o treino

In [8]:
# Criar DataFrames completos com a variável alvo
train_df = X_train.copy()
train_df["num_reg"] = y_train

test_df = X_test.copy()
test_df["num_reg"] = y_test

### Salva os datasets brutos

In [9]:
# Salvar datasets brutos
train_df.to_csv("heart_train.csv", index=False)
test_df.to_csv("heart_test.csv", index=False)
print("Arquivos brutos salvos: heart_train.csv e heart_test.csv")

Arquivos brutos salvos: heart_train.csv e heart_test.csv


## Pré-processamento

In [10]:
# Separando as colunas por grupos numéricos e categóricos
num_cols = ["age", "trestbps", "chol", "thalch", "oldpeak"]
cat_cols = ["sex", "dataset", "cp", "fbs", "restecg", "exang", "slope"]

Antes de treinar modelos, é importante preparar os dados:

* **Numéricas**: normalização ou padronização (MinMaxScaler ou StandardScaler).
* **Categóricas**: codificação (OneHotEncoder ou LabelEncoder).

**1. Variáveis Numéricas: Padronização vs. Normalização**
* **Padronização (StandardScaler)**: Transforma os dados para terem média 0 e desvio padrão 1. Não os limita a um intervalo específico. **(AIXA sensibilidade a Outliers)**
* **Normalização (MinMaxScaler)**: Redimensiona os dados para um intervalo fixo, geralmente entre 0 e 1.**(ALTA sensibilidade a Outliers)**

**2. Variáveis Categóricas: OneHotEncoder vs. LabelEncoder**
Essa escolha é menos sobre o algoritmo e mais sobre a **natureza da variável**. A escolha errada aqui pode seriamente prejudicar o desempenho do seu modelo.

* **OneHotEncoder** - usado para codificar variáveis independentes (features do eixo X). Cria uma nova coluna binária para cada categoria, sem criar uma relação de ordem falsa. Por exemplo: coluna sex=[Male,Female] vira coluna sex_male=[0,1] e coluna sex_female=[0,1]
* **LabelEncoder** - usada normalmente para codificar variável alvo (feature do eixo Y) ou variáveis genuinamente ordinais. Por exemplo: Coluna risco=['baixo', 'médio', 'alto'] vira coluna risco=[0,1,2] 

**Decisão:**
* **Numéricas: padronização (StandardScaler)**.
* **Categóricas: codificação (OneHotEncoder)**.

In [11]:
# Pré-processamento
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first'), cat_cols)
])
# Display 
display(preprocessor)

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['age', 'trestbps', 'chol', 'thalch',
                                  'oldpeak']),
                                ('cat', OneHotEncoder(drop='first'),
                                 ['sex', 'dataset', 'cp', 'fbs', 'restecg',
                                  'exang', 'slope'])])

In [12]:
# Ajustar e transformar os dados de treino
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [13]:
# Converter arrays pré-processados de volta para DataFrame
# Para colunas numéricas, manter nomes originais; para categóricas, criar nomes a partir do encoder
encoded_cat_cols = preprocessor.named_transformers_['cat'].get_feature_names_out(cat_cols)
processed_columns = num_cols + list(encoded_cat_cols)

train_processed_df = pd.DataFrame(X_train_processed, columns=processed_columns)
train_processed_df["num_reag"] = y_train.reset_index(drop=True)

test_processed_df = pd.DataFrame(X_test_processed, columns=processed_columns)
test_processed_df["num_reag"] = y_test.reset_index(drop=True)

### Salva os datasets pré-processados prontos para treinar modelos

In [14]:
# Salvar datasets pré-processados
train_processed_df.to_csv("heart_train_processed.csv", index=False)
test_processed_df.to_csv("heart_test_processed.csv", index=False)
print("Arquivos pré-processados salvos: heart_train_processed.csv e heart_test_processed.csv")

Arquivos pré-processados salvos: heart_train_processed.csv e heart_test_processed.csv


### Salvar o objeto preprocessor (joblib) para aplicar exatamente o mesmo pré-processamento em novos dados.

In [15]:
# Salvar pré-processador para uso futuro
joblib.dump(preprocessor, "preprocessor.joblib")
print("Pré-processador salvo: preprocessor.joblib")

Pré-processador salvo: preprocessor.joblib


In [16]:
# Carregando a extensão watermark
%reload_ext watermark

In [17]:
# Imprimir
# -a (autor do projeto)
# -d (data)
# -t (hora)
# -v (versões do Python e do IPython)
# -m (informações sobre a máquina (hardware e sistema operacional))
#--iversions (versões de pacotes importadas (import))
%watermark -n -a "Patrick F. R. Ribeiro" -d -t -v -m --iversions

Author: Patrick F. R. Ribeiro

Python implementation: CPython
Python version       : 3.13.5
IPython version      : 8.30.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit

numpy  : 1.26.4
sklearn: 1.6.1
pandas : 2.2.3
joblib : 1.4.2

